In [1]:
import polars as pl
import pyodbc
from datetime import date

In [2]:
# import functions.common as fn
import time
start_time = time.time()

### Definición de Rutas

In [3]:
cierre = "202505"
cierre_l = "May25"
cierre_d = date(2025,5,31)

# Carpetas
wd = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = wd + "data/raw/"
wd_data_external = wd + "data/external/"
wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/DWH/"
wd_data_processed_fotos = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/Fotos/"
wd_data_validations = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/validations/"

# Inputs
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'
fl_cohortes_1 = wd_data_raw + f'Cohortes_{cierre}_1 - copia.mdb'
fl_cohortes_2 = wd_data_raw + f'Cohortes_{cierre}_2.mdb'
fl_cohortes_3 = wd_data_raw + f'Cohortes_{cierre}_3.mdb'
fl_cohortes_4 = wd_data_raw + f'Cohortes_{cierre}_4.mdb'
fl_cohortes_5 = wd_data_raw + f'Cohortes_{cierre}_5.mdb'
fl_saldo_mgi = wd_data_raw + f'Saldo_MGI_{cierre}.mdb'
fl_saldo_mgi_fianzas = wd_data_raw + f'Saldo_MGI_80686_{cierre}.mdb'

# Outputs
fl_bd_dwh_saldoymgi_vf = wd_data_processed_dwh + f'parquet/BD_DWH_SaldoyMGI_{cierre}_VF.parquet'
fl_db_dwh_r = wd_data_processed_fotos + f'parquet/DB_DWH_R_{cierre}.parquet'
fl_db_dwh_nr = wd_data_processed_fotos + f'parquet/DB_DWH_NR_{cierre}.parquet'

fl_valida_r = wd_data_validations + f'Valida_R.csv'
fl_valida_nr = wd_data_validations + f'Valida_NR.csv'
fl_valida_agrup_nr = wd_data_validations + f'Valida_Agrup_NR.csv'
fl_valida_agrup_r = wd_data_validations + f'Valida_Agrup_R.csv'


In [4]:
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)
tipo_credito = pl.read_database(query='SELECT * FROM [TIPO_CREDITO]', connection=conn, schema_overrides={'Tipo_Credito_ID': pl.Int32})
conn.close()


### 04 Carga BD

In [5]:
schema_cohortes = {
        'BUCKET': pl.Int16,
        'CONREC_CLAVE': pl.Int32,
        'DESC_INDICADOR': pl.String,
        'Describe_Desrec': pl.String,
        'ESTADO_ID': pl.Int32,
        'ESTRATO_ID': pl.Int16,
        'FECHA_APERTURA': pl.Datetime,
        'FECHA_PRIMER_INCUMPLIMIENTO': pl.Datetime,
        'FECHA_REGISTRO_ALTA': pl.Datetime,
        'INDICADOR_ID': pl.Int32,
        'INTERMEDIARIO_ID': pl.String,
        'NOMBRE_EMPRESA': pl.String,
        'NUMERO_CREDITO': pl.String,
        'PLAZO': pl.Int32,
        'PLAZO_DIAS': pl.Int32,
        'PORCENTAJE_COMISION_GARANTIA': pl.Float32,
        'PORCENTAJE_GARANTIZADO': pl.Float32,
        'PROGRAMA_ID': pl.Int32,
        'PROGRAMA_ORIGINAL': pl.Int32,
        'RAZON_SOCIAL': pl.String,
        'RFC_EMPRESA': pl.String,
        'SECTOR_ID': pl.Int16,
        'TASA_ID': pl.Int16,
        'TIPO_CREDITO_ID': pl.Int32,
        'TIPO_GARANTIA_ID': pl.Int32,
        'VALOR_TASA_INTERES': pl.Float32,
        'MONTO_CREDITO_MN (SUMA)': pl.Float64
}

In [6]:
def read_access(file, table, schema):
    conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + file
    conn = pyodbc.connect(conn_str)
    df = pl.read_database(query=f"SELECT * FROM {table}", connection=conn, schema_overrides=schema)
    conn.close()
    return df


In [7]:
def import_bd_dwh():
    """Importa y pega Cohortes"""

    db_dwh = read_access(fl_cohortes_1, "DATOS", schema_cohortes)

    cohortes = [
        fl_cohortes_2,
        fl_cohortes_3,
        fl_cohortes_4,
        fl_cohortes_5
    ]

    def import_db_dwh_(db_dwh, db):
        aux = read_access(db, "DATOS", schema_cohortes)
        db_dwh = pl.concat([db_dwh, aux], rechunk=True, how='diagonal')
        return db_dwh


    for db in cohortes:
        db_dwh = import_db_dwh_(db_dwh, db)

    return db_dwh

db_dwh = import_bd_dwh()

MemoryError: 

In [ ]:
def genera_dwh_nr_r(db_dwh):
    db_dwh = (db_dwh
            .join(tipo_credito, 
                    how="left", 
                    left_on="TIPO_CREDITO_ID", 
                    right_on="Tipo_Credito_ID"
                )
            )
    
    db_dwh = db_dwh.select(
        pl.col("BUCKET"),
        pl.col("DESC_INDICADOR").alias("Producto"),
        pl.col("ESTADO_ID"),
        pl.col("ESTRATO_ID"),
        pl.col("FECHA_APERTURA").alias("Fecha de Apertura"),
        pl.col("FECHA_PRIMER_INCUMPLIMIENTO"),
        pl.col("FECHA_REGISTRO_ALTA").alias("Fecha Registro Alta"),
        pl.col("INDICADOR_ID").alias("Producto ID"),
        pl.col("INTERMEDIARIO_ID"),
        pl.col("NOMBRE_EMPRESA").alias("Empresa / Acreditado (Descripción)"),
        pl.col("NUMERO_CREDITO"),
        pl.col("PLAZO"),
        pl.col("PLAZO_DIAS").alias("Plazo Días"),
        pl.col("PORCENTAJE_COMISION_GARANTIA").alias("Porcentaje de Comisión Garantia"),
        pl.col("PORCENTAJE_GARANTIZADO").alias("Porcentaje Garantizado"),
        pl.col("PROGRAMA_ID"),
        pl.col("PROGRAMA_ORIGINAL"),
        pl.col("RAZON_SOCIAL").alias("Razón Social (Intermediario)"),
        pl.col("RFC_EMPRESA").alias("RFC Empresa / Acreditado"),
        pl.col("SECTOR_ID"),
        pl.col("TASA_ID"),
        pl.col("TIPO_CREDITO_ID"),
        pl.col("TIPO_GARANTIA_ID"),
        pl.col("VALOR_TASA_INTERES"),
        pl.col("MONTO_CREDITO_MN (SUMA)").alias("Monto _Credito_Mn"),
        pl.col("CONREC_CLAVE"),
        pl.col("Describe_Desrec"),
        pl.col("NR_R")
    )

    db_dwh_r = db_dwh.filter(pl.col("NR_R")=="R")
    db_dwh_nr = db_dwh.filter(pl.col("NR_R")=="NR")

    return db_dwh_r, db_dwh_nr

db_dwh_r, db_dwh_nr = genera_dwh_nr_r(db_dwh)
del(db_dwh)

In [ ]:
def generate_fvto(df):
    result = (df
        .with_columns(
            (pl.col("Fecha de Apertura") +
            pl.duration(days=(365 * pl.col("PLAZO") / 12).round()) + 
            pl.duration(days=(pl.col("Plazo Días").fill_null(0)))
            )
            .alias("test")))
    return result

db_dwh_r = generate_fvto(db_dwh_r)
db_dwh_nr = generate_fvto(db_dwh_nr)

In [ ]:
def genera_tpro_clave(df):
    result = df.with_columns(
        pl.when((pl.col("PROGRAMA_ID")>=32000)&(pl.col("PROGRAMA_ID")<=32100))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID")==3976)&(pl.col("PROGRAMA_ORIGINAL")==31415))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID")==33366)&(pl.col("PROGRAMA_ORIGINAL")==33842))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID").is_in([3536, 3537, 3539, 3542,3544, 3545, 3546,3547,3548,3549,3550, 3553, 3555, 3558,3559, 3560, 3564,3566]))&(pl.col("PROGRAMA_ORIGINAL")==3200))
        .then(pl.col("PROGRAMA_ID"))
        .when(pl.col("PROGRAMA_ORIGINAL")==3999)
        .then("PROGRAMA_ID")
        .otherwise(pl.col("PROGRAMA_ORIGINAL")).alias("TPRO_CLAVE")
    )
    return result

db_dwh_r = genera_tpro_clave(db_dwh_r)
db_dwh_nr = genera_tpro_clave(db_dwh_nr)

In [ ]:
def genera_tipo_persona(df):
    result = df.with_columns(
        pl.when(pl.col("RFC Empresa / Acreditado").str.slice(3, length=1) == "-")
        .then(pl.lit("M"))
        .otherwise(pl.lit("F")).alias("Tipo_Persona")
    )
    return result

db_dwh_r = genera_tipo_persona(db_dwh_r)
db_dwh_nr = genera_tipo_persona(db_dwh_nr)

In [ ]:
def genera_nombrev1(df):
    result = (
        df.with_columns(
            pl.col("Empresa / Acreditado (Descripción)").str.replace("'", "")
                .alias("Nombre_v1")
        )
    ) 
    return result

db_dwh_r = genera_nombrev1(db_dwh_r)
db_dwh_nr = genera_nombrev1(db_dwh_nr)

### 05 Genera prefotos

In [ ]:
# importa catalogos x2
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)

tipo_cambio = pl.read_database(query='SELECT * FROM [TIPO CAMBIO]', connection=conn)
tipo_garantia = pl.read_database(query='SELECT * FROM [TIPO_GARANTIA]', connection=conn, schema_overrides={'Tipo_garantia_ID': pl.Int32})
udis = pl.read_database(query='SELECT * FROM [UDIS]', connection=conn, schema_overrides={'Fecha_Paridad': pl.Datetime})
programa = pl.read_database(query='SELECT * FROM [PROGRAMA]', connection=conn, schema_overrides={'PROGRAMA_ID': pl.Int128})
agrupamiento = pl.read_database(query='SELECT * FROM [AGRUPAMIENTO]', connection=conn)
sfc = pl.read_database(query='SELECT * FROM [SIN FONDOS CONTRAGARANTIA]', connection=conn)

#udis = udis.with_columns(pl.col("Fecha_Paridad").cast(pl.Date))
conn.close()

tipo_cambio.columns = ["Anio", "Mes", "TC"]

tdc = tipo_cambio.filter((pl.col("Anio")==cierre_d.year) & (pl.col("Mes")==cierre_d.month)).select("TC").item()
tdc

19.3957

In [ ]:
# Crea columna Fecha_Consulta x2
def gen_col_fecha_consulta(df):
    result = (
        df.with_columns(
            pl.lit(cierre).alias("Fecha_Consulta")
        )
    )
    return result

db_dwh_r = gen_col_fecha_consulta(db_dwh_r)
db_dwh_nr = gen_col_fecha_consulta(db_dwh_nr)

In [ ]:
# Modifica columna Tipo_Garantia_Id x2
def cor_col_garantia_id(df):
    result = df.with_columns(pl.col("TIPO_GARANTIA_ID").fill_null(999))
    return result

db_dwh_r = cor_col_garantia_id(db_dwh_r)
db_dwh_nr = cor_col_garantia_id(db_dwh_nr)

In [ ]:
# pega catalogos x2
def cruza_catalogos(df):
    """
    Requiere que se hayan importado los catálogos. Se omite pegado de tipo de cambio porque lo pega todo parejo :/ Si se va a usar funciona mejor como escalar.
    
    """

    result = (df
    .join(programa.select(['PROGRAMA_ID', 'AGRUPAMIENTO_ID', 'ESQUEMA', 'SUBESQUEMA']), on="PROGRAMA_ID", how='left')
        .join(agrupamiento, on='AGRUPAMIENTO_ID', how='left')
        .join(udis, left_on="Fecha de Apertura", right_on="Fecha_Paridad", how='left')
        .join(tipo_credito.select(['Tipo_Credito_ID', 'NR_R']), left_on='TIPO_CREDITO_ID', right_on="Tipo_Credito_ID", how='left')
        .join(tipo_garantia.select(['Tipo_garantia_ID', 'CSG']), left_on='TIPO_GARANTIA_ID', right_on='Tipo_garantia_ID', how='left')
        .join(sfc.select(['Intermediario_Id', 'CLAVE_CREDITO', 'FONDOS_CONTRAGARANTIA']), left_on=['INTERMEDIARIO_ID', 'NUMERO_CREDITO'], right_on=['Intermediario_Id', 'CLAVE_CREDITO'], how='left')
        )
     
    # Complementa
    result = (result
        .with_columns(pl.when(pl.col("Monto _Credito_Mn")<=(900000*pl.col("Paridad_Peso")))
                    .then(0).otherwise(1).alias("MM_UDIS"))
        .with_columns(pl.when(pl.col("FONDOS_CONTRAGARANTIA")=="SF")
                    .then(pl.lit("SF")).otherwise(pl.lit("CF")).alias("CSF"))
        )
    
    return result

db_dwh_r = cruza_catalogos(db_dwh_r)
db_dwh_nr = cruza_catalogos(db_dwh_nr)

### 06 Sube Saldos y MGI

In [ ]:
# importa tablas + concatenación
def import_saldos_mgi():

    schema_saldos = {
        "INTERMEDIARIO_ID": pl.String,
        "MONEDA_ID": pl.Int16,
        "NUMERO_CREDITO": pl.String,
        "MONTO_GARANTIZADO (SUMA)": pl.Float64,
        "SALDO_CONTINGENTE (SUMA)": pl.Float64
    }
    df_saldo_mgi = read_access(fl_saldo_mgi, "DATOS", schema_saldos)
    df_saldo_mgi_fianzas = read_access(fl_saldo_mgi_fianzas, "DATOS", schema_saldos)

    result = pl.concat([
        df_saldo_mgi,
        df_saldo_mgi_fianzas
    ], rechunk=True)
    return result

db_dwh_saldoymgi = import_saldos_mgi()

In [ ]:
def genera_concatenado(df):
    result = (df.with_columns(
                (pl.col("INTERMEDIARIO_ID") + pl.col("NUMERO_CREDITO"))
                .alias("Concatenado"))
            )
    return result

def genera_bd_dwh_saldoymgi_vf(df):
    result = genera_concatenado(df)
    result = (
        result.select(
            pl.col("Concatenado"),
            pl.col("INTERMEDIARIO_ID"),
            pl.col("MONEDA_ID"),
            pl.col("NUMERO_CREDITO"), 
            pl.col("MONTO_GARANTIZADO (SUMA)").alias("Monto_Garantizado"),
            pl.col("SALDO_CONTINGENTE (SUMA)").alias("Saldo_Contingente")
        )
    )
    return result 

# Exporta
db_dwh_saldoymgi_vf = genera_bd_dwh_saldoymgi_vf(db_dwh_saldoymgi)

### 07 Genera Saldos

In [ ]:
db_dwh_r = genera_concatenado(db_dwh_r)
db_dwh_nr = genera_concatenado(db_dwh_nr)

In [ ]:
db_dwh_saldoymgi_vf

Concatenado,INTERMEDIARIO_ID,MONEDA_ID,NUMERO_CREDITO,Monto_Garantizado,Saldo_Contingente
str,str,i16,str,f64,f64
"""100001441303695150""","""10000144""",1,"""1303695150""",10258.92,0.0
"""100001442000712164""","""10000144""",1,"""2000712164""",21250.0,0.0
"""100001445000027685""","""10000144""",1,"""5000027685""",83021.09,0.0
"""100001445000030359""","""10000144""",1,"""5000030359""",20670.29,0.0
"""100001445000333372""","""10000144""",1,"""5000333372""",2875.0,0.0
…,…,…,…,…,…
"""4736216161617818""","""473621616""",1,"""1617818""",149761.76,0.0
"""4736216161617930""","""473621616""",1,"""1617930""",25214.37,0.0
"""4736216161621651""","""473621616""",54,"""1621651""",2780.38,0.0


In [ ]:
def genera_dwh_nrr_vf(df, db_dwh_saldoymgi_vf):
    result = (df.join(db_dwh_saldoymgi_vf, how="left", on="Concatenado")
    .with_columns(
        pl.when(pl.col("MONEDA_ID") == 1).then(pl.lit(1))
        .when(pl.col("MONEDA_ID") == 54).then(pl.lit(tdc))
        .alias("TC")
        )
    .with_columns(
        (pl.col("Saldo_Contingente") * pl.col("TC")).alias("Saldo_Contingente_Mn"),
        (pl.col("Monto_Garantizado") * pl.col("TC")).alias("Monto_Garantizado_Mn_Original"),
        (pl.col("Monto _Credito_Mn") * pl.col("Porcentaje Garantizado") / 100).alias("Monto_Garantizado_Mn")
        )
    .drop(["NR_R_right", "INTERMEDIARIO_ID_right", "NUMERO_CREDITO_right"])
    )
    return result

#Falta compara_registros
db_dwh_r = genera_dwh_nrr_vf(db_dwh_r, db_dwh_saldoymgi_vf)
db_dwh_nr = genera_dwh_nrr_vf(db_dwh_nr, db_dwh_saldoymgi_vf)

In [ ]:
valida_agrup_r = (db_dwh_r
 .filter(pl.col("AGRUPAMIENTO").is_null()==True)
 .group_by("Producto")
 .agg(
    pl.col("Saldo_Contingente_Mn").sum()
))
valida_agrup_r

Producto,Saldo_Contingente_Mn
str,f64


In [ ]:
valida_agrup_nr = (db_dwh_nr
 .filter(pl.col("AGRUPAMIENTO").is_null()==True)
 .group_by("Producto")
 .agg(
    pl.col("Saldo_Contingente_Mn").sum()
))
valida_agrup_nr

Producto,Saldo_Contingente_Mn
str,f64


In [ ]:
valida_r = (db_dwh_r
 .group_by("Producto")
 .agg(
    pl.col("Saldo_Contingente_Mn").sum().alias("Saldo_MDP_Suma")
))
valida_r

Producto,Saldo_MDP_Suma
str,f64
"""GARANTIA SECTORIAL""",1.1183e9
"""GARANTIA EMPRESARIAL""",2.3628e10
"""GARANTIAS FEFA""",2.2851e8
"""EMPRESA MEDIANA""",3.4222e8
"""GARANTIA MICROCREDITO""",0.0
"""GARANTIA SUBASTA""",0.0
"""GARANTIA SELECTIVA""",0.0
"""GARANTIA ESPECIAL""",6.7328e7


In [ ]:
valida_nr = (db_dwh_nr
 .group_by("Producto")
 .agg(
    pl.col("Saldo_Contingente_Mn").sum().alias("Saldo_MDP_Suma")
))
valida_nr

Producto,Saldo_MDP_Suma
str,f64
"""GARANTIA SECTORIAL""",1.6590e10
"""GARANTIA FIANZA""",6.7765e9
"""GARANTIA SUBASTA""",438167.378693
"""GARANTIA MICROCREDITO""",0.0
"""GARANTIA SELECTIVA""",0.0
"""EMPRESA MEDIANA""",6.4169e7
"""GARANTIAS FEFA""",1.3752e8
"""GARANTIA EMPRESARIAL""",6.8374e10
"""GARANTIA EMERGENTES""",4.7356e8


In [ ]:
# Exporta Parquets
db_dwh_saldoymgi_vf.write_parquet(fl_bd_dwh_saldoymgi_vf)
db_dwh_nr.write_parquet(fl_db_dwh_nr)
db_dwh_r.write_parquet(fl_db_dwh_r)

# Exporta Validadores
valida_agrup_r.write_csv(fl_valida_agrup_r)
valida_agrup_nr.write_csv(fl_valida_agrup_nr)
valida_r.write_csv(fl_valida_r)
valida_nr.write_csv(fl_valida_nr)

In [ ]:
end_time = time.time()
end_time - start_time

96.50881123542786